In [2]:
import pandas as pd
import numpy as np
import json
import re

/var/folders/r3/t2r07q612snc9xmhh7_6wkqm0000gn/T/ipykernel_88802/3722360228.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Edgelist azione

In [3]:
df_azione = pd.read_csv('../../data/telegram/messaggi_telegram_azione_incel.csv')

/var/folders/r3/t2r07q612snc9xmhh7_6wkqm0000gn/T/ipykernel_88802/4182464802.py:1: DtypeWarning: Columns (4,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_azione = pd.read_csv('../data/telegram/messaggi_telegram_azione_incel.csv')


In [4]:
df_azione

,_,id,peer_id,date,action,out,mentioned,media_unread,silent,post,...,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason
0,MessageService,372042,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-01-21 09:28:49+00:00,"{'_': 'MessageActionGameScore', 'game_id': 120...",False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Message,375671,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-01-31 11:54:08+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 1, 'replies...",NaN,NaN,NaN,NaN,[]
2,MessageService,375672,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-01-31 13:03:58+00:00,"{'_': 'MessageActionChatJoinedByLink', 'invite...",False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Message,375674,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-01-31 13:06:45+00:00,NaN,False,False,False,False,False,...,NaN,"[{'_': 'MessageEntityMention', 'offset': 0, 'l...",NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
4,Message,375681,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-01-31 13:19:01+00:00,NaN,False,False,False,False,False,...,NaN,"[{'_': 'MessageEntityBotCommand', 'offset': 13...",NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36843,Message,413419,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-04-03 10:43:45+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
36844,Message,413420,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-04-03 10:44:54+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
36845,Message,413421,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-04-03 10:53:16+00:00,NaN,False,False,False,False,False,...,NaN,"[{'_': 'MessageEntityMentionName', 'offset': 0...",NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
36846,Message,413422,"{'_': 'PeerChannel', 'channel_id': 1767430683}",2024-04-03 11:39:17+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


## Preprocess 'fwd_from' column to import as JSON

In [5]:
def format_json(stringa):
    for element in re.findall(r''''[^'"]+?'[,:]''', stringa):
        n_element = element.replace('\'','"')
        stringa = stringa.replace(element,n_element)

    stringa = stringa.replace('False','false').replace('None','null')
    return json.loads(stringa)

In [6]:
serie_forward = []
error_count=0
for i in df_azione[df_azione['fwd_from'].notna()]['fwd_from']:
    try: 
        json_row = format_json(i)
        serie_forward.append(json_row)
    except json.JSONDecodeError:
        error_count+=1

print(error_count)

3


In [7]:
len(serie_forward)

727

## Get channel codes

In [8]:
def filter_serie(iterable:list|pd.Series):
    """From series or list to serie of channels
        iterable: list or pandas series
    """
    if type(iterable)==list:
        serie = pd.Series(iterable)

    elif type(iterable)==pd.Series:
        serie = iterable

    else:
        raise TypeError('iterable must be list or series')   
    
    serie = serie.apply(lambda x: x['from_id'])
    mask = serie.apply(lambda x: type(x))==dict
    serie = serie[mask]
    list_of_channels = serie[serie.apply(lambda x: x['_'])=='PeerChannel']
    return list_of_channels.apply(lambda x: x['channel_id'])

In [9]:
serie_codes = filter_serie(serie_forward)

In [10]:
serie_codes = serie_codes.value_counts()\
    .reset_index()\
    .rename(columns={'index':'forward_from'})


In [11]:
serie_codes['forward_to']=1767430683 #id for Azione Incel

In [12]:
first_edgelist_azione = serie_codes[['forward_from','forward_to','count']]

# Edgelist FDB

In [13]:
df_fdb = pd.read_csv('../../data/telegram/messaggi_telegram_fdb.csv')

/var/folders/r3/t2r07q612snc9xmhh7_6wkqm0000gn/T/ipykernel_88802/1859297291.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fdb = pd.read_csv('../data/telegram/messaggi_telegram_fdb.csv')


In [14]:
df_fdb

,_,id,peer_id,date,action,out,mentioned,media_unread,silent,post,...,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason
0,MessageService,326388,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-02-06 09:43:28+00:00,"{'_': 'MessageActionGameScore', 'game_id': 120...",False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Message,327919,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-02-06 11:50:46+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
2,Message,327920,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-02-06 11:50:51+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
3,Message,327921,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-02-06 11:50:58+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
4,Message,327922,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-02-06 11:51:07+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165134,Message,498536,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-04-04 11:49:25+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
165135,Message,498537,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-04-04 11:49:27+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
165136,Message,498538,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-04-04 11:49:39+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]
165137,Message,498539,"{'_': 'PeerChannel', 'channel_id': 2084686607}",2024-04-04 11:49:54+00:00,NaN,False,False,False,False,False,...,NaN,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[]


## Preprocess 'fwd_from' column to import as JSON

In [15]:
def format_json(stringa):
    for element in re.findall(r''''[^'"]+?'[,:]''', stringa):
        n_element = element.replace('\'','"')
        stringa = stringa.replace(element,n_element)

    stringa = stringa.replace('False','false').replace('None','null')
    return json.loads(stringa)

In [16]:
serie_forward = []
error_count= 0
for i in df_fdb[df_fdb['fwd_from'].notna()]['fwd_from']:
    try: 
        json_row = format_json(i)
        serie_forward.append(json_row)
    except json.JSONDecodeError:
        error_count+=1
print(error_count)

0


In [17]:
len(serie_forward)

1710

## Get channel codes

In [18]:
def filter_serie(iterable:list|pd.Series):
    """From series or list to serie of channels
        iterable: list or pandas series
    """
    if type(iterable)==list:
        serie = pd.Series(iterable)

    elif type(iterable)==pd.Series:
        serie = iterable

    else:
        raise TypeError('iterable must be list or series')   
    
    serie = serie.apply(lambda x: x['from_id'])
    mask = serie.apply(lambda x: type(x))==dict
    serie = serie[mask]
    list_of_channels = serie[serie.apply(lambda x: x['_'])=='PeerChannel']
    return list_of_channels.apply(lambda x: x['channel_id'])

In [19]:
serie_codes = filter_serie(serie_forward)

In [20]:
serie_codes = serie_codes.value_counts()\
    .reset_index()\
    .rename(columns={'index':'forward_from'})


In [21]:
serie_codes['forward_to']=2084686607 #code for FDB

In [22]:
first_edgelist_fdb = serie_codes[['forward_from','forward_to','count']]

# Put edgelists together

In [23]:
first_edgelist_total = pd.concat([first_edgelist_azione, first_edgelist_fdb])

In [24]:
first_edgelist_total.to_csv('../first_edgelist/first_edgelist_final.csv', index=False)